# Space Station 🚀
```
           ___
     |     | |
    / \    | |
   |--o|===|-|
   |---|   |N|
  /     \  |A|
 | Snow  | |S|
 | Ex    |=|A|
 |       | | |
 |_______| |_|
  |@| |@|  | |
  *** *** 
 ***********
*************
```



## SnowEx Hackweek 2022

### "Hack"-tronauts
- **Team Leader:** Micah "Sub-naught" Johnson
- Beau Uriona
- Wyatt Reis
- Angela Rienzo
- Danny Hogan
- Jordan Beer
- Ibrahim Olalekan Alabi
- Dylan Craaybeek
- Micah "Prime" Sandusky 
- Ross Palomaki

## Motivation

1. Can we build a pipeline to connect a SNOTEL station's observations to the closest relavent SnowEx data of interest?
2. How does a SNOTEL station represent surrounding measured snow from ASO's snow depth product?
3. What are the main drivers for the surrounding heterogeneity (distance, predominant wind, slope, veg cover, aspect, elevation, etc.)?

## Process
![Diagram showing raster to depth](./figures/raster-depth-diff.jpg)

## Locations

### Map with All Locations
![Show ArcPro figures here](figures/ZoomedOut.png)

### Location close-up with Buffer
![Show ArcPro figures here](figures/ZoomedIn.png)

## Results and Visualizations

### Visualize depth difference between snow depth raster and SNOTEL snow dpeth

In [ ]:
# Show Point map with difference values 

### Some rudimentary stats 

![Show ArcPro figures here](figures/histogram.png)
![Show ArcPro figures here](figures/line_plot.png)

## Next Steps

- Analyze with respect to other terrain variables
- Look at other SnowEx flights and products that align with SNOTEL sites
- Snow poles as "stations"
- Expand to look at any snow depth lidar products
- Comparing to snow model output
